In [1]:
import numpy as np
import open3d as o3d
import matplotlib.pyplot as plt
from matplotlib.colors import LinearSegmentedColormap
from open3d.t.geometry import TriangleMesh
from tqdm import tqdm 
import time

Jupyter environment detected. Enabling Open3D WebVisualizer.
[Open3D INFO] WebRTC GUI backend enabled.
[Open3D INFO] WebRTCWindowSystem: HTTP handshake server disabled.


In [2]:
#Pre-processing
dataname = "/home/chris/Code/PointClouds/data/FLIPscans/MortenPlateTopSuperCleaned500k.ply"
pcd = o3d.io.read_point_cloud(dataname)
pcd_center = pcd.get_center()
pcd.translate(-pcd_center)

#Outlier removal
nn = 16
std_multiplier = 10

filtered_pcd = pcd.remove_statistical_outlier(nn,std_multiplier)
outliers = pcd.select_by_index(filtered_pcd[1], invert = True)
outliers.paint_uniform_color([1,0,0])
filtered_pcd = filtered_pcd[0]

#Downsampling
voxel_size = 0.01
pcd_downsampled = filtered_pcd.voxel_down_sample(voxel_size=voxel_size)
print(f'number of points: {len(pcd_downsampled.points)}')

#Extract normals
nn_distance = np.mean([pcd.compute_nearest_neighbor_distance()])
radius_normals = nn_distance*4

pcd_downsampled.estimate_normals(search_param=o3d.geometry.KDTreeSearchParamHybrid(radius=radius_normals, max_nn=16), fast_normal_computation=True)

pcd_downsampled.paint_uniform_color([0.6,0.6,0.6])

front =  [ 0.0, 0.0, 1.0 ]
lookat = [ 4.155493041143778, 3.3619307090130235, 0.041189902146896884 ]
up =  [ 0.0, 1.0, 0.0 ]
zoom = 0.61999999999999988

pcd = pcd_downsampled

# Multi-order Ransac

max_plane_idx = 1
pt_to_plane_dist = 0.4

segment_models = {}
segments = {}
main_surface_idx = 0
largest_surface_points = 0
rest = pcd

for i in range(max_plane_idx):
    #print(f'Run {i}/{max_plane_idx} started. ', end='')
    colors = plt.get_cmap("tab20")(i)
    segment_models[i], inliers = rest.segment_plane(distance_threshold=pt_to_plane_dist,ransac_n=3,num_iterations=50000)
    segments[i] = rest.select_by_index(inliers)
    if len(segments[i].points) > largest_surface_points:
        largest_surface_points = len(segments[i].points) 
        main_surface_idx = i
    segments[i].paint_uniform_color(list(colors[:3]))
    rest = rest.select_by_index(inliers, invert=True)
    print('Done')

# Gap detection
main_surface_pcd = segments[main_surface_idx]
np.asarray(main_surface_pcd.points)[:, 2] = 0

points = np.asarray(main_surface_pcd.points)
pcd_tree = o3d.geometry.KDTreeFlann(main_surface_pcd)

grid_resolution = 0.1
search_radius = 0.5
neighbor_threshold = 4

min_bound = main_surface_pcd.get_min_bound()
max_bound = main_surface_pcd.get_max_bound()

x_grid = np.arange(min_bound[0], max_bound[0], grid_resolution)
y_grid = np.arange(min_bound[1], max_bound[1], grid_resolution)
z_value = np.mean([min_bound[2], max_bound[2]])

# Generate all query points at once
xv, yv = np.meshgrid(x_grid, y_grid, indexing='ij')
query_points = np.vstack((xv.ravel(), yv.ravel(), np.full_like(xv.ravel(), z_value))).T

gap_points = []

# Efficient neighbor search
for query_point in tqdm(query_points):
    k, idx, _ = pcd_tree.search_radius_vector_3d(query_point, search_radius)
    if k < neighbor_threshold:
        gap_points.append(query_point)

gap_pcd = o3d.geometry.PointCloud()
gap_pcd.points = o3d.utility.Vector3dVector(gap_points)
gap_pcd.paint_uniform_color([1, 0, 0])

#DBScan clustering

eps = 0.30
min_points = 10

labels = np.array(gap_pcd.cluster_dbscan(eps=eps, min_points=min_points, print_progress=True))
max_label = labels.max()
print(f"point cloud has {max_label + 1} clusters")
print(labels[0])
base_cmap = plt.get_cmap("tab20")
color_cycle = [base_cmap(i % 20) for i in range(max_label + 1)]
colors = np.array(color_cycle)[labels]
colors[labels<0] = 0
gap_pcd.colors = o3d.utility.Vector3dVector(colors[:, :3])

#Filter small clusters

points = np.asarray(gap_pcd.points)
colors = np.asarray(gap_pcd.colors)  
point_threshold = 45
mask = np.zeros(len(points), dtype=bool)
unique_labels = np.unique(labels)

filtered_pcd = o3d.geometry.PointCloud()
filtered_clusters = []
filtered_labels_unique = []
filtered_labels_vector = []

#Filter and assign
for label in unique_labels:
    if label != -1:
        cluster_indices = np.where(labels == label)[0]
        if len(cluster_indices) >= point_threshold:
            mask[cluster_indices] = True
            cluster_points = points[cluster_indices]
            filtered_labels_unique.append(label)

for label in filtered_labels_unique:
    cluster_indices = np.where(labels == label)[0]
    cluster_points = points[cluster_indices]
    filtered_clusters.append(cluster_points)
    cluster_labels = np.tile(label, (cluster_points.shape[0], 1))
    filtered_labels_vector.append(cluster_labels)

total_points = np.vstack([cluster for cluster in filtered_clusters])
filtered_labels = np.vstack([cluster_labels for cluster_labels in filtered_labels_vector]).squeeze()

#print(total_points.shape)
#print(filtered_labels.shape)

filtered_pcd.points = o3d.utility.Vector3dVector(total_points)

#Recolor the filtered pcd
max_label = filtered_labels.max()
base_cmap = plt.get_cmap("tab20")
color_cycle = [base_cmap(i % 20) for i in range(max_label + 1)]
colors = np.array(color_cycle)[filtered_labels]
filtered_pcd.colors = o3d.utility.Vector3dVector(colors[:, :3])

o3d.visualization.draw_geometries([filtered_pcd], zoom=zoom, front=front, lookat=lookat, up=up)

number of points: 493486
Done


100%|██████████| 6193050/6193050 [00:39<00:00, 156530.37it/s]


Precompute neighbors.[========================================] 100%
point cloud has 102 clusters======================>] 97%
0


In [18]:
def reset_camera(view_ctl):
    # Set camera view parameters (adjust these as needed)
    view_ctl.set_lookat([0, 0, 0])  # Center of the view
    view_ctl.set_up([0, 1, 0])  # Up direction
    view_ctl.set_front([0, 0, -1])  # Front direction
    view_ctl.set_zoom(1.0)  # Zoom level

visualization = o3d.visualization.VisualizerWithKeyCallback()
visualization.create_window()
visualization.get_render_option().background_color = np.asarray([0.95, 0.95, 0.95])

view_ctl = visualization.get_view_control()
#reset_camera(view_ctl)

class ConcaveHullClusterCallback:
    def __init__(self, cluster, k):
        self.cluster_points_3d = cluster
        self.cluster_pcd = o3d.geometry.PointCloud()
        self.cluster_pcd.points = o3d.utility.Vector3dVector(self.cluster_points_3d)
        self.cluster_colors = np.tile((0.1,0.1,0.1), (self.cluster_points_3d.shape[0], 1))
        self.cluster = np.delete(cluster, 2, 1)
        self.number_of_points = self.cluster.shape[0]
        self.section_cloud = False
        self.grid_size = 3
        if self.number_of_points > 10000:
            self.section_cloud = True
            self.create_sections()
        self.direction_vector = np.array([-1, 0])
        self.deactivated_point_indices = []
        self.deactivated_points = []
        self.grid_points = np.ones(self.cluster.shape[0], dtype=bool)
        self.k = k
        self.k_list = [8, 10, 12]
        self.hull_point_indices_global = []
        self.hull_point_indices_local = []
        self.it = 0
        self.breakloop_it = 10000
        self.hullclosed = False
        self.key_pressed = False
        self.collinearity = False
        self.current_section_index = None
        self.current_section_points = None
        self.current_section_points_global_indices = None
    
    def create_sections(self):
        print('Creating Sections', flush=True)
        x_min, y_min = self.cluster.min(axis=0)
        x_max, y_max = self.cluster.max(axis=0)
        
        print(f'x_max={x_max}, x_min={x_min}, y_max={y_max}, y_min={y_min}')
        print(f'x_diff = {(x_max - x_min)}, y_diff = {(y_max - y_min)}')
        M = round((x_max - x_min) / (3.5 * self.grid_size))
        N = round((y_max - y_min) / (3.5 * self.grid_size))
        print(f'M={M} and N={N}')
        if N == 0: 
            y_intervals = np.linspace(y_min, y_max, 2)
        else: 
            y_intervals = np.linspace(y_min, y_max, 2 * N + 1)

        if M == 0: 
            x_intervals = np.linspace(x_min, x_max, 2)
        else:
            x_intervals = np.linspace(x_min, x_max, 2 * M + 1)

        small_sections = []
        small_section_centers = []
        self.small_section_bboxes = []
        small_section_indices = []

        for i in range(max(2 * N, len(y_intervals) - 1)):
            for j in range(max(2 * M, len(x_intervals) - 1)):
                x_start, x_end = x_intervals[j], x_intervals[j + 1]
                y_start, y_end = y_intervals[i], y_intervals[i + 1]

                x_center = (x_start + x_end) / 2.0
                y_center = (y_start + y_end) / 2.0
                small_section_centers.append([x_center, y_center])
                
                if i == max(2 * N, len(y_intervals) - 1) - 1:
                    y_condition = (self.cluster[:, 1] >= y_start) & (self.cluster[:, 1] <= y_end)
                else:
                    y_condition = (self.cluster[:, 1] >= y_start) & (self.cluster[:, 1] < y_end)
                    
                if j == max(2 * M, len(x_intervals) - 1) - 1:
                    x_condition = (self.cluster[:, 0] >= x_start) & (self.cluster[:, 0] <= x_end)
                else:
                    x_condition = (self.cluster[:, 0] >= x_start) & (self.cluster[:, 0] < x_end)

                section_indices = np.where(x_condition & y_condition)[0]

                small_sections.append(self.cluster[section_indices])
                small_section_indices.append(section_indices)
            
                section_bbox = o3d.geometry.AxisAlignedBoundingBox(min_bound=[x_start, y_start, -0.01],max_bound=[x_end,y_end,0.01])
                section_bbox.color = [0.42,0.53,0.39]
                self.small_section_bboxes.append(section_bbox)
        
        print(f'Created {len(small_sections)} small sections.')

        self.section_centers = []
        self.section_indices_arrays = []
        self.section_points_arrays = []

        if N >= 1 and M >= 1:
            for i in range(min(2 * N - 1, len(y_intervals) - 2)):
                for j in range(min(2 * M - 1, len(x_intervals) - 2)):
                    x_start = x_intervals[j]
                    x_end = x_intervals[j + 2]
                    y_start = y_intervals[i]
                    y_end = y_intervals[i + 2]

                    x_center = (x_start + x_end) / 2.0
                    y_center = (y_start + y_end) / 2.0
                    self.section_centers.append([x_center, y_center])
                    
                    #print(f'Section {i*(2*M-1)+j+1} contains small sections {j+2*M*i}, {j+1+2*M*i}, {j+2*M*(i+1)} and {j+1+2*M*(i+1)}')

                    section_indices = np.concatenate([
                        small_section_indices[j + 2 * M * i],
                        small_section_indices[j + 1 + 2 * M * i],
                        small_section_indices[j + 2 * M * (i + 1)],
                        small_section_indices[j + 1 + 2 * M * (i + 1)]
                    ])
                    
                    self.section_indices_arrays.append(section_indices)
                    self.section_points_arrays.append(self.cluster[section_indices])

                    #print(f'Created section {len(self.section_centers)} with indices: {section_indices}')
                    #print(f'Section points set size: {len(self.section_points_sets[-1])}')

        elif N==0 and M>=1:
            for j in range(2*M-1):
                x_start = x_intervals[j]
                x_end = x_intervals[j+2]
                y_start = y_intervals[0]
                y_end = y_intervals[1]

                x_center = (x_start + x_end) / 2.0
                y_center = (y_start + y_end) / 2.0
                self.section_centers.append([x_center, y_center])

                section_indices = np.concatenate([
                    small_section_indices[j],
                    small_section_indices[j+1],
                ])

                self.section_indices_arrays.append(section_indices)
                self.section_points_arrays.append(self.cluster[section_indices])

                #print(f'Created section {len(self.section_centers)} with indices: {section_indices}')
                #print(f'Section points set size: {len(self.section_points_sets[-1])}')


        elif N>=1 and M==0:
            for i in range(2*N-1):
                x_start = x_intervals[0]
                x_end = x_intervals[1]
                y_start = y_intervals[i]
                y_end = y_intervals[i+2]

                x_center = (x_start + x_end) / 2.0
                y_center = (y_start + y_end) / 2.0
                self.section_centers.append([x_center, y_center])

                section_indices = np.concatenate([
                    small_section_indices[i],
                    small_section_indices[i+1],
                ])

                self.section_indices_arrays.append(section_indices)
                self.section_points_arrays.append(self.cluster[section_indices])

                #print(f'Created section {len(self.section_centers)} with indices: {section_indices}')
                #print(f'Section points set size: {len(self.section_points_sets[-1])}')

        elif N==0 and M==0:
            self.section_cloud = False

        print(f'Total sections created: {len(self.section_centers)}', flush=True)

    def calculate_distances(self, hull_point, active_indices):
        if self.section_cloud:
            return np.sqrt(np.sum(np.square(self.current_section_points[active_indices] - hull_point), axis=1))
        else:
            return np.sqrt(np.sum(np.square(self.cluster[active_indices] - hull_point), axis=1))
    
    def find_closest_section(self, current_hull_point):
        distances = np.sqrt(np.sum(np.square(self.section_centers - current_hull_point), axis=1))
        return np.argsort(distances)[0]
    
    def get_knn(self, current_hull_point):
        if self.section_cloud:
            deactivated_mask = np.isin(self.current_section_points_global_indices, self.deactivated_point_indices)
            active_grid_points = np.logical_and(~deactivated_mask, self.grid_points)
            active_grid_points_indices_local = np.nonzero(active_grid_points)[0]

            distances = self.calculate_distances(current_hull_point, active_grid_points_indices_local)
            sorted_indices = np.argsort(distances)

            neighbor_indices = active_grid_points_indices_local[sorted_indices[:self.k]]
            neighbor_points = self.current_section_points[neighbor_indices]
            return neighbor_indices, neighbor_points
        else:
            inactive_indices = np.zeros(self.grid_points.shape, dtype=bool)
            inactive_indices[self.deactivated_point_indices] = True
            active_grid_points = np.logical_and(~inactive_indices, self.grid_points)
            active_grid_points_indices = np.nonzero(active_grid_points)[0]
            distances = self.calculate_distances(current_hull_point, active_grid_points_indices)
            sorted_indices = np.argsort(distances)
            neighbor_indices = active_grid_points_indices[sorted_indices[:self.k]]
            neighbor_points = self.cluster[neighbor_indices]
            return neighbor_indices, neighbor_points

    def get_points_inside_grid(self, current_hull_point):
        grid_size_2d = np.array([self.grid_size, self.grid_size])
        grid_min = current_hull_point - grid_size_2d / 2
        grid_max = current_hull_point + grid_size_2d / 2
        if self.section_cloud:
            #print(f'Current Section Points: {self.current_section_points}', flush=True)
            self.grid_points = np.all((self.current_section_points >= grid_min) & 
                                      (self.current_section_points <= grid_max), axis=1)
        else:
            self.grid_points = np.all((self.cluster >= grid_min) & (self.cluster <= grid_max), axis=1)

    def paint_pcd(self, vis, neighborhood_indices, current_hull_point_global_index, chosen_neighbor_index):
        colors = self.cluster_colors.copy()

        if self.section_cloud:
            colors[self.current_section_points_global_indices] = (0.42, 0.53, 0.39)
            deactivated_mask = np.isin(self.current_section_points_global_indices, self.deactivated_point_indices)
            active_grid_points = np.logical_and(~deactivated_mask, self.grid_points)
            active_grid_points_indices_local = np.nonzero(active_grid_points)[0]
            global_active_grid_points_indices = self.current_section_points_global_indices[active_grid_points_indices_local]
            
            # Color the active grid points in the section
        else:
            inactive_indices = np.zeros(self.grid_points.shape, dtype=bool)
            inactive_indices[self.deactivated_point_indices] = True
            active_grid_points = np.logical_and(~inactive_indices, self.grid_points)
            global_active_grid_points_indices = np.nonzero(active_grid_points)[0]

        # Color the active grid points
        colors[global_active_grid_points_indices] = (0.3, 0.41, 0.57)

        # Color other points
        past_hull_points_indices = np.asarray(self.hull_point_indices_global)
        colors[neighborhood_indices] = (0.9, 0.1, 0.1)
        colors[past_hull_points_indices] = (1, 0.6, 0.1)
        colors[current_hull_point_global_index] = (0.1, 0.1, 0.9)
        colors[chosen_neighbor_index] = (0.1, 0.6, 0.1)
        
        self.cluster_pcd.colors = o3d.utility.Vector3dVector(colors)
        vis.update_geometry(self.cluster_pcd)
        
        if hasattr(self, 'bbox'):
            vis.remove_geometry(self.bbox, reset_bounding_box=False)
        current_hull_point = np.append(self.cluster[current_hull_point_global_index], 0)
        self.bbox = self.create_grid(current_hull_point)
        vis.add_geometry(self.bbox, reset_bounding_box=False)
        
        if hasattr(self, 'bbox'):
            vis.remove_geometry(self.bbox, reset_bounding_box=False)
        current_hull_point = np.append(self.cluster[current_hull_point_global_index], 0)
        self.bbox = self.create_grid(current_hull_point)
        vis.add_geometry(self.bbox, reset_bounding_box=False)

    def create_grid(self, current_hull_point_3d):
        grid_size_3d = np.array([self.grid_size, self.grid_size, 0.01])
        min_bound = current_hull_point_3d - grid_size_3d/2
        max_bound = current_hull_point_3d + grid_size_3d/2
        grid = o3d.geometry.AxisAlignedBoundingBox(min_bound=min_bound, max_bound=max_bound)
        grid.color = (1,0,0)
        return grid

    def find_lowest_point(self):
        lowest_y = np.amin(self.cluster[:,1])
        lowest_y_indices = np.where(self.cluster[:, 1] == lowest_y)[0]

        if lowest_y_indices.shape[0] == 1:
            lowest_point_index = lowest_y_indices[0]
            lowest_point = self.cluster[lowest_point_index]
        elif lowest_y_indices.shape[0] > 1:
            lowest_y_points = self.cluster[lowest_y_indices]
            lowest_x = np.amin(lowest_y_points[:, 0])
            lowest_point_index = np.where((self.cluster[:,0] == lowest_x) & (self.cluster[:,1] == lowest_y))[0][0]
            lowest_point = np.array([lowest_x, lowest_y])
        return lowest_point, lowest_point_index
    
    def unit_vector(self, vector):
        return vector/np.linalg.norm(vector)
    
    def ccw(self,A,B,C):
        return (C[1]-A[1])*(B[0]-A[0]) > (B[1]-A[1])*(C[0]-A[0])

    def check_parallelism(self, vector_alpha, vector_beta):
        determinant = vector_alpha[0] * vector_beta[1] - vector_alpha[1] * vector_beta[0]
        return np.isclose(determinant, 0)
    
    def check_collinearity(self, A, B, C):
        area = A[0]*(B[1]-C[1]) + B[0]*(C[1]-A[1]) + C[0]*(A[1]-B[1])
        if area==0:
            return True
        return False

    def intersect_with_collinearity(self, A, B, C, D):
        if np.array_equal(A, C) or np.array_equal(A, D) or np.array_equal(B, C) or np.array_equal(B, D):
            return False
        if self.check_collinearity(A, C, D) or self.check_collinearity(B, C, D):
            return False
        return self.ccw(A, C, D) != self.ccw(B, C, D) and self.ccw(A, B, C) != self.ccw(A, B, D)
    
    def intersect_without_collinearity(self, A, B, C, D):
        if np.array_equal(A, C) or np.array_equal(A, D) or np.array_equal(B, C) or np.array_equal(B, D):
            return False
        return self.ccw(A, C, D) != self.ccw(B, C, D) and self.ccw(A, B, C) != self.ccw(A, B, D)

    def find_all_indices(self, target_list, query_value):
        return [i for i, x in enumerate(target_list) if x == query_value]
    
    def are_consecutive(self, index1, index2):
        return abs(index1 - index2) == 1
    
    def choose_neighbor(self, current_hull_point, neighbors):
        vectors = neighbors - current_hull_point
        dot_products = np.dot(vectors, self.direction_vector)
        cross_products = np.cross(self.direction_vector, vectors)
        norms = np.linalg.norm(vectors, axis=1)
        cos_angles = np.clip(dot_products / norms, -1.0, 1.0)
        angles = np.arccos(cos_angles)

        right_turn_indices = np.where(cross_products <= 0)[0]
        left_turn_indices = np.where(cross_products > 0)[0]

        if self.it > 3:
            # Separate the 180-degree turns
            threshold = np.deg2rad(179.0)  # Set a threshold for near 180-degree turns
            right_turn_angles = angles[right_turn_indices]
            left_turn_angles = angles[left_turn_indices]

            right_turn_180_indices = right_turn_indices[right_turn_angles >= threshold]
            right_turn_non_180_indices = right_turn_indices[right_turn_angles < threshold]
            left_turn_180_indices = left_turn_indices[left_turn_angles >= threshold]
            left_turn_non_180_indices = left_turn_indices[left_turn_angles < threshold]

            # Sort non-180-degree turns
            sorted_right_turn_non_180_indices = right_turn_non_180_indices[np.argsort(-right_turn_angles[right_turn_angles < threshold])]
            sorted_left_turn_non_180_indices = left_turn_non_180_indices[np.argsort(left_turn_angles[left_turn_angles < threshold])]

            # Combine all indices
            combined_indices = np.concatenate([sorted_right_turn_non_180_indices, sorted_left_turn_non_180_indices, right_turn_180_indices, left_turn_180_indices])
        else:
            right_turn_angles = angles[right_turn_indices]
            left_turn_angles = angles[left_turn_indices]
            sorted_right_turn_indices = right_turn_indices[np.argsort(-right_turn_angles)]
            sorted_left_turn_indices = left_turn_indices[np.argsort(left_turn_angles)]
            combined_indices = np.concatenate([sorted_right_turn_indices, sorted_left_turn_indices])
        
        if self.section_cloud:
            grid_hull_points_indices = [idx for idx in self.hull_point_indices_local if self.grid_points[idx]]
        else:
            grid_hull_points_indices = [idx for idx in self.hull_point_indices_global if self.grid_points[idx]]

        for index in combined_indices:
            chosen_neighbor = neighbors[index]
            C = current_hull_point
            D = chosen_neighbor
            intersects = False

            for i in range(1, len(grid_hull_points_indices)):
                A_index = grid_hull_points_indices[i - 1]
                B_index = grid_hull_points_indices[i]
                
                if self.section_cloud:
                    A_indices = self.find_all_indices(self.hull_point_indices_local, A_index)
                    B_indices = self.find_all_indices(self.hull_point_indices_local, B_index)
                else:
                    A_indices = self.find_all_indices(self.hull_point_indices_global, A_index)
                    B_indices = self.find_all_indices(self.hull_point_indices_global, B_index)

                is_consecutive = any(self.are_consecutive(A_idx, B_idx) for A_idx in A_indices for B_idx in B_indices)

                if is_consecutive:
                    if self.section_cloud:
                        A = self.current_section_points[A_index]
                        B = self.current_section_points[B_index]
                    else:
                        A = self.cluster[A_index]
                        B = self.cluster[B_index]

                    vector_alpha = B - A
                    vector_beta = D - C
                    if self.collinearity == False:
                        if self.intersect_without_collinearity(A, B, C, D) and not self.check_parallelism(vector_alpha, vector_beta):
                            intersects = True
                            break
                    else:
                        if self.intersect_with_collinearity(A, B, C, D) and not self.check_parallelism(vector_alpha, vector_beta):
                            intersects = True
                            break

            if not intersects:
                new_direction_vector = self.unit_vector(chosen_neighbor - current_hull_point)
                self.direction_vector = new_direction_vector
                return chosen_neighbor

        return None

    def increase_k(self, current_hull_point, neighbors):
        for z in range(1, len(self.k_list)):
            self.k = self.k_list[z]
            print(f'Increasing k to {self.k}', flush=True)
            _, neighbors = self.get_knn(current_hull_point)
            chosen_neighbor = self.choose_neighbor(current_hull_point, neighbors)
            if chosen_neighbor is None:
                continue
            else:
                return chosen_neighbor
            
    def enable_collinearity(self, current_hull_point, neighbors):
        print('Enabling collinearity')
        self.collinearity = True
        for z in range(len(self.k_list)):
            self.k = self.k_list[z]
            print(f'Trying with k: {self.k}', flush=True)
            _, neighbors = self.get_knn(current_hull_point)
            chosen_neighbor = self.choose_neighbor(current_hull_point, neighbors)
            if chosen_neighbor is None:
                continue
            else:
                self.collinearity = False
                return chosen_neighbor
        self.collinearity = False

    def concave_hull_iteration(self, vis):
        if self.key_pressed:
            return
        self.key_pressed = True
        self.k = self.k_list[0]
        if self.it == 0:
            self.lowest_point, self.lowest_point_index = self.find_lowest_point()
            #print(f'Lowest point = {self.lowest_point}', flush=True)
            #print(f'Deactivating point with index: {self.lowest_point_index}', flush=True)

            self.hull_point_indices_global.append(self.lowest_point_index)

            initial_colors = self.cluster_colors.copy()
            initial_colors[self.lowest_point_index] = (0.1,0.5,0.1)
            self.cluster_pcd.colors = o3d.utility.Vector3dVector(initial_colors)
            vis.add_geometry(self.cluster_pcd)
            if self.section_cloud:
                for section_bbox in self.small_section_bboxes:
                    vis.add_geometry(section_bbox)
                self.current_section_index = self.find_closest_section(self.lowest_point)
                print(f'Closest section to lowest point is Section {self.current_section_index + 1}', flush=True)
                self.current_section_points_global_indices = self.section_indices_arrays[self.current_section_index]
                self.current_section_points = self.section_points_arrays[self.current_section_index]
                self.hull_point_indices_local = [np.where(self.current_section_points_global_indices == idx)[0][0]
                                                  for idx in self.hull_point_indices_global if idx in 
                                                  self.current_section_points_global_indices]

            self.it += 1
            self.key_pressed = False
        else:
            for j in range(5):
                #print(f'Iteration: {self.it}', flush=True, end='\r')
                #print(25*'-')
                if self.section_cloud and self.hull_point_indices_local:
                    #print(f'number of local hull_points = {len(self.hull_point_indices_local)}', flush=True)
                    #print(f'number of global hull_points = {len(self.hull_point_indices_global)}', flush=True)
                    current_hull_point_global_index = self.hull_point_indices_global[-1]
                    current_hull_point_local_index = self.hull_point_indices_local[-1]
                    current_hull_point = self.current_section_points[current_hull_point_local_index]
                else:
                    current_hull_point_global_index = self.hull_point_indices_global[-1]
                    current_hull_point = self.cluster[current_hull_point_global_index]

                self.deactivated_point_indices.append(current_hull_point_global_index)
                self.deactivated_points.append(current_hull_point)
                
                if self.section_cloud:
                    if self.it%4==0:
                        new_section_index = self.find_closest_section(current_hull_point)
                        if new_section_index != self.current_section_index:
                            #print('Changing sections!', flush=True)
                            self.current_section_index = new_section_index
                            self.current_section_points_global_indices = self.section_indices_arrays[self.current_section_index]
                            self.current_section_points = self.section_points_arrays[self.current_section_index]
                            self.hull_point_indices_local = [np.where(self.current_section_points_global_indices == idx)[0][0]
                                                    for idx in self.hull_point_indices_global if idx in self.current_section_points_global_indices]
                            #print(f'found this number of hull points in the new section: {len(self.hull_point_indices_local)}', flush=True)

                # print(f'Current hull point index: {current_hull_point_global_index}', flush=True)
                # print(f'All hull point indices so far: {self.hull_point_indices_global}', flush=True)
                # print(f'Direction vector: {self.direction_vector}', flush=True)

                if self.it == self.breakloop_it:
                    print('Time to stop this madness', flush=True)
                    return
                
                self.get_points_inside_grid(current_hull_point)
                #print(f'Grid points mask size: {self.grid_points.shape}', flush=True)
                #print(f'Number of grid points: {np.sum(self.grid_points)}', flush=True)

                #local_active_grid_points_indices = np.nonzero(self.grid_points)[0]
                #print(f'Local active grid points indices: {local_active_grid_points_indices}', flush=True)
                #neighborhood_indices, neighbors = self.get_knn(current_hull_point)
                if self.section_cloud:
                    neighborhood_indices_local, neighbors = self.get_knn(current_hull_point)
                    neighborhood_indices_global = self.current_section_points_global_indices[neighborhood_indices_local]
                else:
                    neighborhood_indices_global, neighbors = self.get_knn(current_hull_point)
                #print(f'Neighborhood indices: {neighborhood_indices}', flush=True)
                chosen_neighbor = self.choose_neighbor(current_hull_point, neighbors)
                if chosen_neighbor is None:
                    chosen_neighbor = self.increase_k(current_hull_point, neighbors)
                if chosen_neighbor is None:
                    chosen_neighbor = self.enable_collinearity(current_hull_point, neighbors)
                if chosen_neighbor is None:
                    print('Couldnt find neighbor or find lowest point, closing loop.', flush=True)
                    return
                #print(f'Chosen neighbor: {chosen_neighbor}',flush=True)

                if self.section_cloud:
                    chosen_neighbor_index_local = np.where((self.current_section_points[:, 0] == chosen_neighbor[0]) & (self.current_section_points[:, 1] == chosen_neighbor[1]))[0][0]
                    chosen_neighbor_index_global = self.current_section_points_global_indices[chosen_neighbor_index_local]
                    self.hull_point_indices_local.append(chosen_neighbor_index_local)
                else:
                    chosen_neighbor_index_global = np.where((self.cluster[:, 0] == chosen_neighbor[0]) & (self.cluster[:, 1] == chosen_neighbor[1]))[0][0]

                self.hull_point_indices_global.append(chosen_neighbor_index_global)

                #print(f'Chosen neighbor index: {chosen_neighbor_index}',flush=True)
                if chosen_neighbor_index_global == self.lowest_point_index:
                    print('\nSuccess!',flush=True)
                    return

                if self.it >= int(2*self.k):
                    del self.deactivated_point_indices[0]
                self.it +=1
            
            self.paint_pcd(vis, neighborhood_indices_global, current_hull_point_global_index, chosen_neighbor_index_global)
            self.key_pressed = False

# cluster_points = filtered_clusters[0]
cluster_points = np.asarray(main_surface_pcd.points)
print(f'Number of points in cluster: {cluster_points.shape[0]}')
state = ConcaveHullClusterCallback(cluster_points, k=8)
visualization.register_key_callback(262, state.concave_hull_iteration)

visualization.run()
visualization.destroy_window()

Number of points in cluster: 493354
Creating Sections
x_max=106.30784559335098, x_min=-98.63145440664903, y_max=154.28989616476207, y_min=-147.81010383523795
x_diff = 204.9393, y_diff = 302.1
M=20 and N=29
Created 2320 small sections.
Total sections created: 2223
Closest section to lowest point is Section 8

Success!


In [4]:
# import numpy as np

# # x_min = 0 
# # x_max = 100
# y_min = 20
# y_max = 30

# # N = 2
# # M = 10

# #x_intervals = np.linspace(x_min, x_max, 2 * M + 1)
# y_intervals = np.linspace(y_min, y_max, 2)

# #print(x_intervals)
# print(y_intervals)